In [42]:
import os
import openai
import json
import random
import re
import collections
import math

In [43]:
#Load API keys saved in keys.txt
keyfile = open("keys.txt", "r")
line = keyfile.readline()
line = line.strip()
openai.api_key = line


In [44]:
#DOCS: https://beta.openai.com/docs/api-reference?lang=python

#Engines:
#davinci
#curie
#babbage
#ada
# response = openai.Completion.create(
#     engine="davinci",
#     temperature = 0.0,
#     top_p = 1,
#     prompt="This is a test",
#     max_tokens=50,
#     stop='\n'
#     )

In [45]:
# print(response)

In [46]:
# response_dict = response.to_dict()
# text = response_dict["choices"][0]["text"]

In [47]:
# print(text)

In [48]:
def loadDataset(datasetPath):
    with open(datasetPath, 'r', encoding='utf-8', errors='ignore') as djson:
        examples = json.load(djson)
        dataset = []
        for e in examples:
            #Filter instances with no intent
            if(e['rewritten_intent']):
                x = {}
                x["question"] = e["rewritten_intent"]
                x["truth"] = e["snippet"]
                dataset.append(x)
            
        return dataset

In [49]:
def format_response(response):
    response_dict = response.to_dict()
    text = response_dict["choices"][0]["text"]
    return text

In [78]:
def open_ai_predict(prompt, question):
    #DOCS: https://beta.openai.com/docs/api-reference?lang=python
    #Engines:
    #davinci
    #curie
    #babbage
    #ada
    prompt += question
    prompt += "\n"
    prompt += "Output:"
    print(prompt)
    response = openai.Completion.create(
        engine="ada",
        temperature = 0,
        top_p = 1,
        prompt=prompt,
        max_tokens=50,
        stop='\n'
        )
    text = format_response(response)
    return text

In [79]:
prompts = ["""Input: format number of spaces between strings `Python`, `:` and `Very Good` to be `20`
Output: print('%*s : %*s' % (20, 'Python', 20, 'Very Good'))
###
Input: Remove all items from a dictionary `myDict` whose values are `42`
Output: {key: val for key, val in list(myDict.items()) if val != 42}
###
Input: Convert string '03:55' into datetime.time object
Output: datetime.datetime.strptime('03:55', '%H:%M').time()
###
Input: find intersection data between series `s1` and series `s2`
Output: s1.intersection(s2)
###
Input: get the average of a list values for each key in dictionary `d`)
Output: [(i, sum(j) / len(j)) for i, j in list(d.items())]
###
Input: """,
"""
Input: create a list `listofzeros` of `n` zeros
Output: listofzeros = [0] * n
###
Input: Remove all items from a dictionary `myDict` whose values are `42`
Output: {key: val for key, val in list(myDict.items()) if val != 42}
###
Input: Get duplicate elements from list a_val
Output: [value for value in a_val if value in a_val]
###
Input: find intersection data between series `s1` and series `s2`
Output: s1.intersection(s2)
###
Input: get the average of a list values for each key in dictionary `d`)
Output: [(i, sum(j) / len(j)) for i, j in list(d.items())]
###
Input:  get elements from list `a_val`, that have a field `n` value 30
Output:  [x for x in a_val if x.n == 30]
###
Input: """,
           """
Input: create a list `listofzeros` of `n` zeros
Output: listofzeros = [0] * n
###
Input: Remove all items from a dictionary `myDict` whose values are `42`
Output: {key: val for key, val in list(myDict.items()) if val != 42}
###
Input: """,
"""
Input: run script 'hello.py' with argument 'htmlfilename.htm' on terminal using python executable
Output: subprocess.call(['python.exe', 'hello.py', 'htmlfilename.htm'])
###
Input:  Create new DataFrame object by merging columns \"key\" of  dataframes `split_df` and `csv_df` and rename the columns from dataframes `split_df` and `csv_df` with suffix `_left` and `_right` respectively
Output:  pd.merge(split_df, csv_df, on=['key'], suffixes=('_left', '_right'))
###
Input: encode binary string 'your string' to base64 code
Output: base64.b64encode(bytes('your string', 'utf-8'))
###
Input: """
]

In [80]:
prompt = prompts[3]

In [81]:
dataset = loadDataset("./conala-corpus/conala-train.json")
random.seed(1000)
random.shuffle(dataset)
dataset = dataset[0:100]

In [82]:
dataset

[{'question': "sort json `ips_data` by a key 'data_two'",
  'truth': "sorted_list_of_keyvalues = sorted(list(ips_data.items()), key=item[1]['data_two'])"},
 {'question': "What's the best way to search for a Python dictionary value in a list of dictionaries?",
  'truth': "any(d['site'] == 'Superuser' for d in data)"},
 {'question': 'Get all the second values from a list of lists `A`',
  'truth': '[row[1] for row in A]'},
 {'question': 'decode url-encoded string `some_string` to its character equivalents',
  'truth': 'urllib.parse.unquote(urllib.parse.unquote(some_string))'},
 {'question': 'append string `foo` to list `list`',
  'truth': "list.append('foo')"},
 {'question': 'access an arbitrary value from dictionary `dict`',
  'truth': 'next(iter(list(dict.values())))'},
 {'question': 'Calling an external command "ls -l"',
  'truth': 'from subprocess import call'},
 {'question': 'convert string of bytes `y\\xcc\\xa6\\xbb` into an int',
  'truth': "struct.unpack('<L', 'y\\xcc\\xa6\\xbb')[

In [83]:
# predicted = open_ai_predict(prompt, dataset[14]["question"])

In [84]:
def predict_all(prompt, dataset):
    results = []
    for data in dataset:
        results.append(open_ai_predict(prompt, data["question"]))
    return results

In [85]:
results = predict_all(prompt, dataset)


Input: run script 'hello.py' with argument 'htmlfilename.htm' on terminal using python executable
Output: subprocess.call(['python.exe', 'hello.py', 'htmlfilename.htm'])
###
Input:  Create new DataFrame object by merging columns "key" of  dataframes `split_df` and `csv_df` and rename the columns from dataframes `split_df` and `csv_df` with suffix `_left` and `_right` respectively
Output:  pd.merge(split_df, csv_df, on=['key'], suffixes=('_left', '_right'))
###
Input: encode binary string 'your string' to base64 code
Output: base64.b64encode(bytes('your string', 'utf-8'))
###
Input: sort json `ips_data` by a key 'data_two'
Output:

Input: run script 'hello.py' with argument 'htmlfilename.htm' on terminal using python executable
Output: subprocess.call(['python.exe', 'hello.py', 'htmlfilename.htm'])
###
Input:  Create new DataFrame object by merging columns "key" of  dataframes `split_df` and `csv_df` and rename the columns from dataframes `split_df` and `csv_df` with suffix `_left` and


Input: run script 'hello.py' with argument 'htmlfilename.htm' on terminal using python executable
Output: subprocess.call(['python.exe', 'hello.py', 'htmlfilename.htm'])
###
Input:  Create new DataFrame object by merging columns "key" of  dataframes `split_df` and `csv_df` and rename the columns from dataframes `split_df` and `csv_df` with suffix `_left` and `_right` respectively
Output:  pd.merge(split_df, csv_df, on=['key'], suffixes=('_left', '_right'))
###
Input: encode binary string 'your string' to base64 code
Output: base64.b64encode(bytes('your string', 'utf-8'))
###
Input: get the ASCII value of a character u'あ' as an int
Output:

Input: run script 'hello.py' with argument 'htmlfilename.htm' on terminal using python executable
Output: subprocess.call(['python.exe', 'hello.py', 'htmlfilename.htm'])
###
Input:  Create new DataFrame object by merging columns "key" of  dataframes `split_df` and `csv_df` and rename the columns from dataframes `split_df` and `csv_df` with suffix `_


Input: run script 'hello.py' with argument 'htmlfilename.htm' on terminal using python executable
Output: subprocess.call(['python.exe', 'hello.py', 'htmlfilename.htm'])
###
Input:  Create new DataFrame object by merging columns "key" of  dataframes `split_df` and `csv_df` and rename the columns from dataframes `split_df` and `csv_df` with suffix `_left` and `_right` respectively
Output:  pd.merge(split_df, csv_df, on=['key'], suffixes=('_left', '_right'))
###
Input: encode binary string 'your string' to base64 code
Output: base64.b64encode(bytes('your string', 'utf-8'))
###
Input: Copy list `old_list` and name it `new_list`
Output:

Input: run script 'hello.py' with argument 'htmlfilename.htm' on terminal using python executable
Output: subprocess.call(['python.exe', 'hello.py', 'htmlfilename.htm'])
###
Input:  Create new DataFrame object by merging columns "key" of  dataframes `split_df` and `csv_df` and rename the columns from dataframes `split_df` and `csv_df` with suffix `_left` 


Input: run script 'hello.py' with argument 'htmlfilename.htm' on terminal using python executable
Output: subprocess.call(['python.exe', 'hello.py', 'htmlfilename.htm'])
###
Input:  Create new DataFrame object by merging columns "key" of  dataframes `split_df` and `csv_df` and rename the columns from dataframes `split_df` and `csv_df` with suffix `_left` and `_right` respectively
Output:  pd.merge(split_df, csv_df, on=['key'], suffixes=('_left', '_right'))
###
Input: encode binary string 'your string' to base64 code
Output: base64.b64encode(bytes('your string', 'utf-8'))
###
Input: sum of squares values in a list `l`
Output:

Input: run script 'hello.py' with argument 'htmlfilename.htm' on terminal using python executable
Output: subprocess.call(['python.exe', 'hello.py', 'htmlfilename.htm'])
###
Input:  Create new DataFrame object by merging columns "key" of  dataframes `split_df` and `csv_df` and rename the columns from dataframes `split_df` and `csv_df` with suffix `_left` and `_ri


Input: run script 'hello.py' with argument 'htmlfilename.htm' on terminal using python executable
Output: subprocess.call(['python.exe', 'hello.py', 'htmlfilename.htm'])
###
Input:  Create new DataFrame object by merging columns "key" of  dataframes `split_df` and `csv_df` and rename the columns from dataframes `split_df` and `csv_df` with suffix `_left` and `_right` respectively
Output:  pd.merge(split_df, csv_df, on=['key'], suffixes=('_left', '_right'))
###
Input: encode binary string 'your string' to base64 code
Output: base64.b64encode(bytes('your string', 'utf-8'))
###
Input: plot point marker '.' on series `ts`
Output:

Input: run script 'hello.py' with argument 'htmlfilename.htm' on terminal using python executable
Output: subprocess.call(['python.exe', 'hello.py', 'htmlfilename.htm'])
###
Input:  Create new DataFrame object by merging columns "key" of  dataframes `split_df` and `csv_df` and rename the columns from dataframes `split_df` and `csv_df` with suffix `_left` and `_r


Input: run script 'hello.py' with argument 'htmlfilename.htm' on terminal using python executable
Output: subprocess.call(['python.exe', 'hello.py', 'htmlfilename.htm'])
###
Input:  Create new DataFrame object by merging columns "key" of  dataframes `split_df` and `csv_df` and rename the columns from dataframes `split_df` and `csv_df` with suffix `_left` and `_right` respectively
Output:  pd.merge(split_df, csv_df, on=['key'], suffixes=('_left', '_right'))
###
Input: encode binary string 'your string' to base64 code
Output: base64.b64encode(bytes('your string', 'utf-8'))
###
Input: python sum of ascii values of all characters in a string `string`
Output:

Input: run script 'hello.py' with argument 'htmlfilename.htm' on terminal using python executable
Output: subprocess.call(['python.exe', 'hello.py', 'htmlfilename.htm'])
###
Input:  Create new DataFrame object by merging columns "key" of  dataframes `split_df` and `csv_df` and rename the columns from dataframes `split_df` and `csv_df


Input: run script 'hello.py' with argument 'htmlfilename.htm' on terminal using python executable
Output: subprocess.call(['python.exe', 'hello.py', 'htmlfilename.htm'])
###
Input:  Create new DataFrame object by merging columns "key" of  dataframes `split_df` and `csv_df` and rename the columns from dataframes `split_df` and `csv_df` with suffix `_left` and `_right` respectively
Output:  pd.merge(split_df, csv_df, on=['key'], suffixes=('_left', '_right'))
###
Input: encode binary string 'your string' to base64 code
Output: base64.b64encode(bytes('your string', 'utf-8'))
###
Input: declare an array `variable`
Output:

Input: run script 'hello.py' with argument 'htmlfilename.htm' on terminal using python executable
Output: subprocess.call(['python.exe', 'hello.py', 'htmlfilename.htm'])
###
Input:  Create new DataFrame object by merging columns "key" of  dataframes `split_df` and `csv_df` and rename the columns from dataframes `split_df` and `csv_df` with suffix `_left` and `_right` res


Input: run script 'hello.py' with argument 'htmlfilename.htm' on terminal using python executable
Output: subprocess.call(['python.exe', 'hello.py', 'htmlfilename.htm'])
###
Input:  Create new DataFrame object by merging columns "key" of  dataframes `split_df` and `csv_df` and rename the columns from dataframes `split_df` and `csv_df` with suffix `_left` and `_right` respectively
Output:  pd.merge(split_df, csv_df, on=['key'], suffixes=('_left', '_right'))
###
Input: encode binary string 'your string' to base64 code
Output: base64.b64encode(bytes('your string', 'utf-8'))
###
Input: find the string matches within parenthesis from a string `s` using regex
Output:

Input: run script 'hello.py' with argument 'htmlfilename.htm' on terminal using python executable
Output: subprocess.call(['python.exe', 'hello.py', 'htmlfilename.htm'])
###
Input:  Create new DataFrame object by merging columns "key" of  dataframes `split_df` and `csv_df` and rename the columns from dataframes `split_df` and 

In [86]:
print(json.dumps([x['truth'] for x in dataset]))

["sorted_list_of_keyvalues = sorted(list(ips_data.items()), key=item[1]['data_two'])", "any(d['site'] == 'Superuser' for d in data)", "[row[1] for row in A]", "urllib.parse.unquote(urllib.parse.unquote(some_string))", "list.append('foo')", "next(iter(list(dict.values())))", "from subprocess import call", "struct.unpack('<L', 'y\\xcc\\xa6\\xbb')[0]", "print(sorted(L, key=lambda x: int(x.split('.')[2])))", "round(3.0005, 3)", "sorted(list(y.items()), key=lambda x: (x[1], x[0]), reverse=True)", "a[np.in1d(a, b)]", "some_func(*params)", "ord('\\u3042')", "hash(pformat(a)) == hash(pformat(b))", "pd.merge(y, x, on='k')[['a', 'b', 'y']]", "ast.literal_eval('(1,2,3,4)')", "[(x / y) for x, y in zip(a, b)]", "if url.endswith('.com'):\n    url = url[:(-4)]", "result = db.engine.execute('<sql here>')", "f = []\nfor (dirpath, dirnames, filenames) in walk(mypath):\n    f.extend(filenames)\n    break", "glob.glob('*')", "[(y - x) for x, y in zip(L, L[1:])]", "sys.exit()", "print([u for v in [[i, i] f

In [87]:
print(json.dumps(results))

[" sort json `ips_data` by a key 'data_two'", " find_dictionary(list(dictionary))", " [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25", " base64.decode(url('some_string')).decode(some_string)", " append(list('foo', 'bar'))", " dict.get(key=['value'], value=['value'], key=['value'], value=['value'], key=['value'], value=['value'], key=['value'], value=['value", "", " int(bytes('y\\xcc\\xa6\\xbb', 'utf-8'))", " sort(['L', 'A', 'B', 'C'], reverse=True)", " round(3.0005, 3)", " sort(y)", " align(b, a)", " some_func(params)", " int(u'\u3042')", " hash(a, b)", " pd.merge(x, y, on=['a', 'b'], suffixes=('y', 'a'))", " (1,2,3,4)", " a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24", " remove the substring \".com\" from the end of the string `url`", " sqlalchemy.sql.queue.execute(['<sql here>', '<sql here>'])", " list(mypath)", " list all files in a current directory", " list(subtract(list(list(list(list(list(li

## Bleu score calculation from CoNaLa baseline repo

In [88]:
""" The tokenizer that we use for code submissions, from Wang Ling et al., Latent Predictor Networks for Code Generation (2016)
    @param code: string containing a code snippet
    @return: list of code tokens
"""
def tokenize_for_bleu_eval(code):
    code = re.sub(r'([^A-Za-z0-9_])', r' \1 ', code)
    code = re.sub(r'([a-z])([A-Z])', r'\1 \2', code)
    code = re.sub(r'\s+', ' ', code)
    code = code.replace('"', '`')
    code = code.replace('\'', '`')
    tokens = [t for t in code.split(' ') if t]

    return tokens

In [89]:
def _get_ngrams(segment, max_order):
    """Extracts all n-grams upto a given maximum order from an input segment.
    Args:
    segment: text segment from which n-grams will be extracted.
    max_order: maximum length in tokens of the n-grams returned by this
        methods.
    Returns:
    The Counter containing all n-grams upto max_order in segment
    with a count of how many times each n-gram occurred.
    """
    ngram_counts = collections.Counter()
    for order in range(1, max_order + 1):
        for i in range(0, len(segment) - order + 1):
            ngram = tuple(segment[i:i+order])
            ngram_counts[ngram] += 1
    return ngram_counts

def compute_bleu(reference_corpus, translation_corpus, max_order=4,
                 smooth=False):
    """Computes BLEU score of translated segments against one or more references.
    Args:
    reference_corpus: list of lists of references for each translation. Each
        reference should be tokenized into a list of tokens.
    translation_corpus: list of translations to score. Each translation
        should be tokenized into a list of tokens.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
    Returns:
    3-Tuple with the BLEU score, n-gram precisions, geometric mean of n-gram
    precisions and brevity penalty.
    """
    matches_by_order = [0] * max_order
    possible_matches_by_order = [0] * max_order
    reference_length = 0
    translation_length = 0
    for (references, translation) in zip(reference_corpus,
                                       translation_corpus):
        reference_length += min(len(r) for r in references)
        translation_length += len(translation)

        merged_ref_ngram_counts = collections.Counter()
        for reference in references:
            merged_ref_ngram_counts |= _get_ngrams(reference, max_order)
        translation_ngram_counts = _get_ngrams(translation, max_order)
        overlap = translation_ngram_counts & merged_ref_ngram_counts
        for ngram in overlap:
            matches_by_order[len(ngram)-1] += overlap[ngram]
        for order in range(1, max_order+1):
            possible_matches = len(translation) - order + 1
            if possible_matches > 0:
                possible_matches_by_order[order-1] += possible_matches

    precisions = [0] * max_order
    for i in range(0, max_order):
        if smooth:
            precisions[i] = ((matches_by_order[i] + 1.) /
                           (possible_matches_by_order[i] + 1.))
        else:
            if possible_matches_by_order[i] > 0:
                precisions[i] = (float(matches_by_order[i]) /
                                 possible_matches_by_order[i])
            else:
                precisions[i] = 0.0

    if min(precisions) > 0:
        p_log_sum = sum((1. / max_order) * math.log(p) for p in precisions)
        geo_mean = math.exp(p_log_sum)
    else:
        geo_mean = 0.0

    ratio = float(translation_length) / reference_length

    if ratio > 1.0:
        bp = 1.
    else:
        bp = math.exp(1 - 1. / ratio)

    bleu = geo_mean * bp

    return (bleu, precisions, bp, ratio, translation_length, reference_length)


In [90]:
def calculate_bleu(dataset, results):
    c_ref = [tokenize_for_bleu_eval(s['truth']) for s in dataset]
    c_hyp = [tokenize_for_bleu_eval(s) for s in results]
        
    bleu_tup = compute_bleu([[x] for x in c_ref], c_hyp, smooth=False)
    bleu = bleu_tup[0]
    return bleu * 100

In [91]:
calculate_bleu(dataset, results)

15.9904777579471

In [184]:
#Save and load our results
def save_results(dataset, results, title):
    d1 = json.dumps(dataset)
    d2 = json.dumps(results)
    
    p1 = os.path.join("results/"+title)
    if not os.path.exists(p1):
        try:
            os.makedirs(p1)
        except OSError as exc: # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise
            
    with open(os.path.join("results/"+title, "dataset.txt"), 'w+') as f:
        json.dump(dataset, f)
    with open(os.path.join("results/"+title, "results.txt"), 'w+') as f:
        json.dump(results, f)
        
def load_results(title):
    d1 = []
    d2 = []
    with open(os.path.join("results/"+title, "dataset.txt"), 'r') as f:
        d1 = json.load(f)
    with open(os.path.join("results/"+title, "results.txt"), 'r') as f:
        d2 = json.load(f)
    return(d1, d2)

In [185]:
save_results(dataset, results, "prompt1_curie_100")
d1, d2 = load_results("prompt1_curie_100")

In [186]:
calculate_bleu(d1, d2)

18.24671310583173